In [0]:
import requests
import json
import datetime

CREATE CONNECTION IF NOT EXISTS earthquake_conn
TYPE HTTP
OPTIONS (
  host = 'https://earthquake.usgs.gov',
  port = '443',
  base_path = '/earthquakes/fee/v1.0/',
  bearer_token = 'na'
)

In [0]:
url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson'

In [0]:
#Usar la conexion creada anteriormente
from databricks.sdk import WorkspaceClient
w = WorkspaceClient()
conn = w.connections.get("earquake_test")
base_url = f"{conn.options['host']}{conn.options['base_path']}"


In [0]:
dbutils.widgets.text('catalog_name','youtube_de')
catalog_name = dbutils.widgets.get('catalog_name')

In [0]:
%python
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql("USE SCHEMA bronze")
spark.sql("CREATE VOLUME IF NOT EXISTS volume_earthquake")


In [0]:
url2 = f"{base_url}/summary/all_day.geojson"
response = requests.get(url2)

if response.status_code != 200:
    raise Exception(f"Error consumiento la data de {url2}")
else:
    data = response.json()
    current_date = datetime.datetime.now().strftime("%Y-%m-%d")
    dbutils.fs.put(
        f"/Volumes/{catalog_name}/bronze/volume_earthquake/earthquake_data_{current_date}.json",
        json.dumps(data),
        overwrite=True,
    )